In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [5]:
feats_type = ['mfcc', 'mfcc_d1','mfcc_d2','engy','envp','mel','sflux','sroll','loudness']
DF = pd.read_csv('./data/csvs/pooled_subject_response_acoustic_feats_data.csv')  


In [7]:
DF['RT'] = DF['t_resp'] - DF['t_end'] # make a response time column
DF['label-response'] = DF['label'] - DF['response'] # make a correct response column
indx = []
indx.append(DF.loc[DF['session'] == 'TEST'].index.to_numpy())
indx.append(np.where(DF["label-response"].values ==0)[0])
indx.append(DF.loc[DF['lang'] == 0].index.to_numpy()) # chinese
indx.append(DF.loc[DF['lang'] == 1].index.to_numpy()) # english
indx.append(np.where(DF["RT"].values >.200)[0])
indx.append(np.where(DF["RT"].values <2)[0])
indx_correct = []
# collect all indices for chinese+correct
indx_correct.append(np.array(list(set(indx[0]) & set(indx[1]) & set(indx[2]) & set(indx[4]) & set(indx[5]))))
# collect all indices for english+correct
indx_correct.append(np.array(list(set(indx[0]) & set(indx[1]) & set(indx[3]) & set(indx[4]) & set(indx[5]))))

In [14]:
# do a linear regression fit on chinese+correct: RT vs features
DF_subset = DF.loc[indx_correct[0], :]
RT = DF_subset["RT"].values
features = DF_subset[feats_type].values

In [15]:
data_x = sm.add_constant(features,prepend=False)
data_y = 10*np.log10(RT.copy()*1000)
mod = sm.OLS(data_y, data_x)

In [16]:
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     5.026
Date:                Fri, 27 Mar 2020   Prob (F-statistic):           9.96e-07
Time:                        23:13:05   Log-Likelihood:                -4235.2
No. Observations:                2076   AIC:                             8490.
Df Residuals:                    2066   BIC:                             8547.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.3422      0.139     -2.470      0.0